# Selecting columns 3: selecting multiple columns
By the end of this lecture you will be able to:
- select columns based on a regex
- select columns based on dtype
- do set operations with multiple selections

Polars has two ways for selecting multiple columns:
- the expression API with `pl.col` or `pl.all`
- the selectors API with polars selectors

We see both of these in this lecture.

To use the selectors API we typically import it as `cs`

In [ ]:
import polars as pl
import polars.selectors as cs

In [ ]:
csvFile = "../data/titanic.csv"

In [ ]:
df = pl.read_csv(csvFile)
df.head(3)

## Using the expression API
### Selecting all columns from a `DataFrame`

We can select all columns by replacing `pl.col` with `pl.all`

In [ ]:
(
    df
    .select(
        pl.all()
    )
    .head(3)
)

We can select all but a subset of columns with the `exclude` expression

In [ ]:
(
    df
    .select(
        pl.exclude(['PassengerId','Survived','Pclass'])
    )
    .head(3)
)

This is a shorthand for `pl.all().exclude(...)`

### Selecting columns with a regex
We can select columns with a regex - if the regex starts with `^` and ends with `$`.

The following regex looks for columns starting with `P` and uses the regex *wildcard* `.*` to show `P` can be followed by any characters.

In [ ]:
(
    df
    .select(
        "^P.*$"
    )
    .head(3)
)

We can pass this regex to `pl.col` to apply transformations to these columns. In this example we take the `max` of each column

In [ ]:
(
    df
    .select(
        pl.col("^P.*$").max()
    )
    .head(3)
)

### Selecting columns based on dtype
We can select all of the columns that have a particular dtype by passing the dtype to `pl.col`.

Here we select all the string columns with `pl.Utf8` - the string dtype object

In [ ]:
(
    df
    .select(
        pl.col(pl.Utf8)
    )
    .head(3)
)

We can also pass a list of dtypes to `pl.col`. In this case we select both 64-bit integer and float columns

In [ ]:
(
    df
    .select(
        pl.col([pl.Int64,pl.Float64])
    )
    .head(3)
)

As a shorthand we can also use shortcuts such as `pl.NUMERIC_DTYPES` to select all numeric dtypes

In [ ]:
(
    df
    .select(
        pl.col(pl.NUMERIC_DTYPES)
    )
    .head(3)
)

The `pl.NUMERIC_DTYPES` is really just a sequence of the underlying dtypes - we can see this if we print it

In [ ]:
pl.NUMERIC_DTYPES

There are a number of other generic dtype objects - we find these from the `pl` namespace below

In [ ]:
[el for el in dir(pl) if "_DTYPES" in el]

## Using the selectors API
The selectors API aims to make selecting multiple columns less verbose. 

For simple cases it replicates using the expression API. For example to select all columns we use `cs.all`

In [ ]:
(
    df
    .select(
        cs.all()
    )
    .head(3)
)

We can also do selection by position with `first` or `last`

In [ ]:
(
    df
    .select(
        cs.first()
    )
    .head(3)
)

The output of a selector is a standard Polars expression so we can follow it up with standard expression chaining

In [ ]:
(
    df
    .select(
        cs.all().max()
    )
)

The selectors API works well in lazy mode and for streaming queries just as expressions do.

We can select columns by groups of dtype - including a group of all integer and floating point dtypes with `cs.numeric`

In [ ]:
(
    df
    .select(
        cs.numeric()
    )
    .head(3)
)

We can select by name - in this example with a `~` operator to exclude the names listed

In [ ]:
(
    df
    .select(
        ~cs.by_name("Pclass","Age")
    )
    .head(3)
)

As a simpler alternative to the regex example we saw earlier we can use string methods such as:
- `contains`
- `starts_with`
- `end_with`
- `matches`

In this example we select all columns beginning with P

In [ ]:
(
    df
    .select(
        cs.starts_with("P")
    )
    .head(3)
)

We can apply an OR condition by passing multiple strings

In [ ]:
(
    df
    .select(
        cs.starts_with("P","A")
    )
    .head(3)
).columns

With the `matches` method we can pass a regex without the `^` and `$` we need for the expression API

In [ ]:
(
    df
    .select(
        cs.matches("Age|Fare")
    )
    .head(3)
)

### Intersection of selectors

To do an intersection of selector conditions we use the `&` operator to say both conditions must be fulfilled.

In this example we look for **numeric** columns that **contain A** in the column name

In [ ]:
(
    df
    .select(
        cs.numeric() & cs.contains("A") 
    )
    .head(3)
)

### Union of selectors
To do a union operation we use the `|` operator to say at least one of the conditions must be satisfied

In [ ]:
(
    df
    .select(
        cs.string() | cs.contains("P") 
    )
    .head(3)
)

### Difference of selectors
To do a difference operation we use a minus operator `-`.

In this example we select all string columns other than any column beginning with T

In [ ]:
(
    df
    .select(
        cs.string() - cs.starts_with("T") 
    )
    .head(3)
)

# Exercises

In the exercises you will develop your understanding of:
- selecting all columns from a `DataFrame`
- excluding columns from a selection
- selecting columns with a regex
- selecting columns with a dtype

### Exercise 1

Select all columns from the `DataFrame` and sort each column. Use the expression API

In [ ]:
(
    pl.read_csv(csvFile)
    <blank>
    .head(3)
)

### Exercise 2
Select all columns from the `DataFrame` with the exception of the `PassengerId` column. Use the expression API

In [ ]:
(
    pl.read_csv(csvFile)
    <blank>
    .head(3)
)

Use the selector API to select all string columns other than the `Name` column

In [ ]:
(
    pl.read_csv(csvFile)
    <blank>
    .head(3)
)

### Exercise 3
Select all columns from the `DataFrame` that start with `S` or `N`. Use the selector API

In [ ]:
(
    pl.read_csv(csvFile)
    <blank>
    .head(3)
)

### Exercise 4
Select all the columns with 64-bit floating point dtype with the expression API

Hint: the 64-bit floating point dtype is `pl.Float64`

In [ ]:
(
    pl.read_csv(csvFile)
    <blank>
    .head(3)
)

Select all columns with a numeric dtype using the selector API

In [ ]:
(
    pl.read_csv(csvFile)
    <blank>
    .head(3)
)

### Exercise 5
Convert the following Pandas code (that I've seen in the wild!) to Polars

Looping over columns in Polars is to be avoided at all costs. 

Convert this Pandas code with a loop over the columns to Polars code using the Expression API.

In the loop we create a dictionary `maxDict` with the column names and maximum values

In [ ]:
import pandas as pd
import numpy as np
df = pl.read_csv(csvFile)
dfPandas = df.to_pandas()

# Convert this code below to Polars in the following cell
maxDict = {}
for col in dfPandas.columns:
    if dfPandas[col].dtype == np.float64:
        maxDict[col] = [dfPandas[col].max()]
pd.DataFrame(maxDict)

In [ ]:
(
    pl.read_csv(csvFile)
     <blank>
)

## Solutions

### Solution to Exercise 1
Select all columns from the `DataFrame` and sort each column. Use the expression API

In [ ]:
(
    pl.read_csv(csvFile)
    .select(
        pl.all().sort()
    )
    .head(2)
)

### Solution to Exercise 2
Select all columns from the `DataFrame` with the exception of the `PassengerId` column. Use the expression API

In [ ]:
(
    pl.read_csv(csvFile)
    .select(
        pl.all().exclude('PassengerId')
    )
    .head(3)
)

Use the selector API to select all string columns other than the `Name` column

In [ ]:
(
    pl.read_csv(csvFile)
    .select(
        cs.string() - cs.by_name("Name")
    )
    .head(3)
)

### Solution to Exercise 3
Select all columns from the `DataFrame` that start with `S` or `N`. Use the selector API

In [ ]:
(
    pl.read_csv(csvFile)
    .select(
        cs.starts_with("S","N")
    )
    .head(3)
)

### Solution to Exercise 4
Select all the columns with 64-bit floating point dtype with the expression API

Hint: the 64-bit floating point dtype is `pl.Float64`

In [ ]:
(
    pl.read_csv(csvFile)
    .select(
        pl.col(pl.Float64)
    )
    .head(3)
)

Select all columns with a numeric dtype using the selector API

In [ ]:
(
    pl.read_csv(csvFile)
    .select(
        cs.numeric()
    )
    .head(3)
)

### Solution to Exercise 5
Convert the following Pandas code to Polars
```python
import pandas as pd
import numpy as np
df = pl.read_csv(csvFile)
dfPandas = df.to_pandas()

# Convert this code below to Polars in the following cell
maxDict = {}
for col in dfPandas.columns:
    if dfPandas[col].dtype == np.float64:
        maxDict[col] = [dfPandas[col].max()]
pd.DataFrame(maxDict)
```

In [ ]:
(
    pl.read_csv(csvFile)
    .select(
        pl.col(pl.Float64).max()
    )
)

Note that there is a better way to do this in Pandas (I just don't see this so often in the wild!)

In [ ]:
df_pandas = df.to_pandas()
df_pandas.select_dtypes("float").max()